In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [231]:
#week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]
#week_day_Apr_list = [2, 9, 16, 23, 30]
week_day_Apr_list = [30]

In [232]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [233]:
len(link_day_minute_Apr_list)

2880

In [234]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 120)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [235]:
np.size(x, 0), np.size(x, 1)

(24, 120)

In [236]:
x[:,:2]

matrix([[  2157.45537733,   2960.16519517],
        [  4306.70818827,   3421.77815583],
        [  4079.9119397 ,   3200.71751413],
        [  2567.866728  ,   3525.02764552],
        [  7179.19964789,   2962.14165586],
        [  2336.03687122,   2032.39882149],
        [  1765.71169437,   2610.61776883],
        [  4602.89122423,   5268.42549987],
        [ 13120.29402794,  10277.03087482],
        [  5019.18207941,   8430.19015583],
        [ 13694.6039845 ,  14676.96704387],
        [  6357.07521285,   4591.79057429],
        [  2466.3036143 ,   1879.06107701],
        [  2776.95187381,   6290.31072903],
        [  6580.97750232,   7107.78389566],
        [  3768.74499255,   4476.81609559],
        [  1180.35950847,    954.46170617],
        [  1638.0609911 ,    972.58974311],
        [  2313.5533953 ,   2281.88149522],
        [ 10311.48667294,  12495.61028052],
        [  6069.26493451,   6662.16339767],
        [  6371.785336  ,   8024.18657806],
        [  2424.72699972,   2154

In [237]:
np.size(A,0), np.size(A,1)

(24, 140)

In [238]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e-03, 5e+00]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.74866205e+06  0.00000000e+00  6.54e+03 1.28e+02  9.38e+05     0s
   1   3.10080037e+06 -3.89559829e+06  6.54e-03 1.14e-13  1.25e+05     0s
   2   3.38585159e+05 -6.78726847e+05  6.54e-09 5.68e-14  1.82e+04     0s
   3  -1.47682879e+04 -1.56630229e+05  2.27e-13 1.28e-13  2.5

In [239]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_30_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [240]:
lam_list

[9.990809450009463e-09,
 2543.700306843245,
 1.0966104845962986e-08,
 1780.3511270837432,
 1876.290081438854,
 3.313682179789147e-05,
 3.013323356432945e-06,
 1713.0198718898844,
 338.75069086538286,
 3941.4325142741336,
 306.468139423627,
 310.48398440787014,
 3.352534404400115e-05,
 3.457356841591805e-05,
 1.688277902572432e-05,
 1.7247396679695784e-05,
 8.902198386909633e-06,
 6941.096574061657,
 10215.68757890814,
 4120.423339480798,
 3.319223486046921e-05,
 0.0002298248833049743,
 4784.601556360022,
 2202.873907757392,
 0.0003080800722412014,
 0.000310212052279657,
 1.7792242690498755e-05,
 1644.5048493374377,
 0.00037839885530471795,
 0.000202031154513449,
 2496.286574362216,
 0.0009663495909557618,
 6341.98631149177,
 1.875083624942875e-05,
 1.879318946817598e-05,
 7.258167711190339e-05,
 7.61786099923549e-05,
 2798.256840536934,
 3.3368332521674024e-05,
 4723.7870284474175,
 5361.1313943814275,
 825.9082839461495,
 4488.953676219058,
 844.7850625051484,
 9.545903911886353e-05,
